In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Step -1
#importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Set to display all the columns of the dataset
pd.set_option("display.max_columns",None)

#Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
#Step -2
#Load the data
cdata=pd.read_csv(r"/content/drive/MyDrive/newdata after EDA.csv",header=0)
#Copy to back_up file
cdata_bk=cdata.copy()
#Display first 5 records
cdata.head()

,age(in years),gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi,bmi_category,pulse_pressure,map
0,50.391781,2,168.0,62.0,110.0,80.0,1,1,0,0,1,0,21.967120,0,30.0,89.9
1,55.419178,1,156.0,85.0,140.0,90.0,3,1,0,0,1,1,34.927679,1,50.0,106.5
2,51.663014,1,165.0,64.0,130.0,70.0,3,1,0,0,0,1,23.507805,0,60.0,89.8
3,48.282192,2,169.0,82.0,150.0,100.0,1,1,0,0,1,1,28.710479,2,50.0,116.5
4,60.038356,1,151.0,67.0,120.0,80.0,2,2,0,0,0,0,29.384676,2,40.0,93.2


In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
import pickle
from joblib import dump

# Pelican Optimization Algorithm (POA) with convergence check
def pelican_optimization(population_size, num_iterations, lower_bounds, upper_bounds, dimension, objective_function, X_train, y_train, X_val, y_val):
    positions = np.random.uniform(low=lower_bounds, high=upper_bounds, size=(population_size, dimension))
    fitness = np.array([objective_function(pos, X_train, y_train, X_val, y_val) for pos in positions])
    convergence_curve = np.zeros(num_iterations)

    best_fitness = np.inf
    best_candidate_solution = None

    for iteration in range(num_iterations):
        prey_position = np.random.uniform(low=lower_bounds, high=upper_bounds, size=dimension)

        for i in range(population_size):
            new_solution = positions[i]

            current_fitness = objective_function(new_solution, X_train, y_train, X_val, y_val)

            if current_fitness < best_fitness:
                best_fitness = current_fitness
                best_candidate_solution = new_solution.copy()

        convergence_curve[iteration] = best_fitness

        if iteration > 0 and convergence_curve[iteration] == convergence_curve[iteration - 1]:
            break  # Break the loop if convergence criterion is met

    return best_candidate_solution, best_fitness, convergence_curve

def objective_function(params, X_train, y_train, X_val, y_val):
    learning_rate, max_depth, n_estimators = params

    model = LGBMClassifier(
        learning_rate=learning_rate,
        max_depth=int(max_depth),
        n_estimators=int(n_estimators),
        subsample_freq=10,
        subsample=0.8,
        reg_lambda=0.0,
        reg_alpha=0.1,
        num_leaves=30,
        min_split_gain=0.1,
        min_child_weight=0.001,
        min_child_samples=20,
        max_bin=500,
        colsample_bytree=0.8
    )
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='accuracy'
    )

    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

def save_model(model, filename):
    # Save model using pickle
    with open(filename + '.pkl', 'wb') as f:
        pickle.dump(model, f)
    # Save model using joblib
    dump(model, filename + '.joblib')

# Load your dataset
# Separate features and target variable
X = cdata.drop('cardio', axis=1)
y = cdata['cardio']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

population_size = 20
num_iterations = 100
lower_bounds = [0.001, 2, 500]  # Adjust as needed
upper_bounds = [0.1, 20, 1000]  # Adjust as needed
dimension = len(lower_bounds)

best_candidate_solution, best_fitness, convergence_curve = pelican_optimization(
    population_size, num_iterations, lower_bounds, upper_bounds, dimension, objective_function, X_train, y_train, X_test, y_test
)

best_learning_rate, best_max_depth, best_n_estimators = best_candidate_solution

final_model = LGBMClassifier(
    n_estimators=int(best_n_estimators),
    learning_rate=best_learning_rate,
    max_depth=int(best_max_depth),
    subsample_freq=10,
    subsample=0.8,
    reg_lambda=0.0,
    reg_alpha=0.1,
    num_leaves=30,
    min_split_gain=0.1,
    min_child_weight=0.001,
    min_child_samples=20,
    max_bin=500,
    colsample_bytree=0.8
)

final_model.fit(X_train, y_train)
save_model(final_model, "final_model")


Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [14]:
from google.colab import drive

def save_model(model, filename):
    # Save model using pickle
    with open('/content/drive/My Drive/' + filename + '.pkl', 'wb') as f:
        pickle.dump(model, f)
    # Save model using joblib
    dump(model, '/content/drive/My Drive/' + filename + '.joblib')
    drive.flush_and_unmount()

# Example usage
save_model(final_model, "final_model")
